In [2]:
import numpy as np
from keras import utils
from keras.optimizers import Adam
from tensorflow.keras.models import load_model
from keras.datasets import cifar10
from keras.datasets import cifar100

In [3]:
dataset1 = cifar10
(x_train1, y_train1), (x_test1, y_test1) = dataset1.load_data() #original size train(xxx, 28, 28),test(xxx)
       
#translate the data to image
x_train1 = x_train1.astype("float32")
x_test1 = x_test1.astype("float32")


#normalization
x_train1 /= 255
x_test1 /= 255

vgg_10 = load_model('Model/vgg_cifar10.h5')#load lenet5 model trained with mnist dataset 
vgg_10.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy']) 

In [23]:
beta = 0.04079
sample(x_test1,y_test1,vgg_10,beta)

Beta is reduced by 0.0001, the new beta is 0.04069
Beta is reduced by 0.0001, the new beta is 0.040589999999999994
Beta is reduced by 0.0001, the new beta is 0.04048999999999999
Beta is reduced by 0.0001, the new beta is 0.04038999999999999
Beta is reduced by 0.0001, the new beta is 0.040289999999999986
Beta is reduced by 0.0001, the new beta is 0.04018999999999998
Beta is reduced by 0.0001, the new beta is 0.04008999999999998
Beta is reduced by 0.0001, the new beta is 0.03998999999999998
Beta is reduced by 0.0001, the new beta is 0.039889999999999974
Beta is reduced by 0.0001, the new beta is 0.03978999999999997
Beta is reduced by 0.0001, the new beta is 0.03968999999999997
Beta is reduced by 0.0001, the new beta is 0.039589999999999965
Beta is reduced by 0.0001, the new beta is 0.03948999999999996
Beta is reduced by 0.0001, the new beta is 0.03938999999999996
Beta is reduced by 0.0001, the new beta is 0.03928999999999996
Beta is reduced by 0.0001, the new beta is 0.039189999999999954

0.014089999999999832

In [35]:
dataset2 = cifar100
(x_train2, y_train2), (x_test2, y_test2) = dataset2.load_data() #original size train(xxx, 28, 28),test(xxx)

img = x_test2[0]        
#translate the data to image
x_train2 = x_train2.astype("float32")
x_test2 = x_test2.astype("float32")


#normalization
x_train2 /= 255
x_test2 /= 255

vgg_100 = load_model('Model/vgg_cifar100.h5')#load lenet5 model trained with mnist dataset 
vgg_100.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy']) 


In [34]:
beta2 = 0.02820
sample(x_test2,y_test2,vgg_100,beta2)

Beta is reduced by 0.0001, the new beta is 0.04069
Beta is reduced by 0.0001, the new beta is 0.040589999999999994
Beta is reduced by 0.0001, the new beta is 0.04048999999999999
Beta is reduced by 0.0001, the new beta is 0.04038999999999999
Beta is reduced by 0.0001, the new beta is 0.040289999999999986
Beta is reduced by 0.0001, the new beta is 0.04018999999999998
Beta is reduced by 0.0001, the new beta is 0.04008999999999998
Beta is reduced by 0.0001, the new beta is 0.03998999999999998
Beta is reduced by 0.0001, the new beta is 0.039889999999999974
Beta is reduced by 0.0001, the new beta is 0.03978999999999997
Beta is reduced by 0.0001, the new beta is 0.03968999999999997
Beta is reduced by 0.0001, the new beta is 0.039589999999999965
Beta is reduced by 0.0001, the new beta is 0.03948999999999996
Beta is reduced by 0.0001, the new beta is 0.03938999999999996
Beta is reduced by 0.0001, the new beta is 0.03928999999999996
Beta is reduced by 0.0001, the new beta is 0.039189999999999954

0.028489999999999745

In [8]:
def sample(x_test,y_test,model, beta):
    mean = np.zeros(3)  
    cov = np.eye(3)
    
    for i in range(x_test.shape[0]):
        count = 0
        while True: 
            p = np.random.multivariate_normal(mean, cov,(32,32)) #sample formula
            neighbour = p[0] + x_test[i]*255
            neighbour = neighbour.reshape(-1,32,32,3)
            neighbour /= 255
            distance=np.linalg.norm(p[0]) 
            distance /= 255
            #print(distance)
            if (distance < beta or distance == beta):
                output = model.predict(neighbour,batch_size=128)
                proba = softmax(output)
                pred = np.argmax(proba)
                label = y_test[i]
                
                if(pred == label):
                    count+=1

            if(count == N):
                #print(i,"satisfied the given beta")
                break
            elif (j < N+499): 
                continue 
            elif (j == N+499):
                beta = beta-0.0001
                print("Beta is reduced by 0.0001, the new beta is",beta) 
                break
                
    print("global robustness formula satisfied")     
    print("max safe distance:",beta)
    return beta

def softmax(x):
    e_x = np.exp(x-np.max(x))
    return e_x / e_x.sum()